# Weather Forecast Data

This notebook is used to extract the weather forecast data from the Open-Meteo API ([Link](https://open-meteo.com/en/docs)). First, we get the coordinates of the locations of which we want to have the forecasts. This is done via a SQL query to an existing database on our Azure SQL Server. Having the coordinates, we call the weather forecast API and get hourly data for the next day for each location, i.e. we receive 24 data entries per location.

The weather forecast consists of several data points like temperature, precipitation probability, wind speed or cloud coverage. In total, we retrieve 19 meteorological data points per location and hour. The received data is then ingested to a separate table in our SQL database. From there, it can be joined with other data sources for further analysis and evaluation.

##  Libraries and settings

In [2]:
# Import required libraries

import json
import os
import pandas as pd
import math
import time

# connect to Open-Meteo API
import openmeteo_requests
import requests_cache
from retry_requests import retry

# connection to SQL database
import pymssql
from sqlalchemy import create_engine, text

In [3]:
# Get current working directory
print(os.getcwd())

c:\Users\twips\Documents\GitHub\Wanderwege\notebooks


## Retrieve hiking route data from database

In [4]:
# Load configuration from config/db_config.json
with open('../config/db_config.json', 'r') as f:
    db_config = json.load(f)

# Get database credentials
server = db_config['server']
database = db_config['database']
db_user = db_config['db_user']
db_password = db_config['db_password']

In [6]:
# Establish connection
conn = pymssql.connect(server, db_user, db_password, database)

# Execute SQL query
query = "SELECT * FROM dbo.OVRP_HikingRoutes"
cursor = conn.cursor()
cursor.execute(query)

# Fetch all the rows from the executed query
rows = cursor.fetchall()

# Fetch the column names from the cursor description
columns = [col[0] for col in cursor.description]

# Store the results in a Pandas DataFrame
df = pd.DataFrame(rows, columns=columns)

# Close the connection
conn.close()

### Data inspection (hiking routes)

In [7]:
# Print the shape of the dataframe
df.shape

(15309, 5)

In [8]:
# Print the first 5 rows of the dataframe
df.head()

,id,name,lat,lon,timestamp_apicall
0,22614,Nationalpark Wanderroute 15 (Munt la Schera),46.650143,10.230199,2024-09-25 21:32:08
1,103607,Wanderwege SG,47.430977,9.620170,2024-09-25 21:32:08
2,112830,Uetliberg - Uetliberg Uto Kulm,47.351168,8.489780,2024-09-25 21:32:08
3,112831,Folenweid - Baldern,47.329124,8.500726,2024-09-25 21:32:08
4,112833,Felsenegg - Balderen,47.315244,8.505056,2024-09-25 21:32:08


In [9]:
# Print the data types of the columns
df.dtypes

id                            int64
name                         object
lat                         float64
lon                         float64
timestamp_apicall    datetime64[ns]
dtype: object

In [10]:
# Describe the dataframe (columns "lat" and "lon")
df[["lat", "lon"]].describe()

,lat,lon
count,15309.000000,15309.000000
mean,47.038307,8.272102
std,0.368350,0.748453
min,45.880236,6.094391
25%,46.861716,7.687536
50%,47.081667,8.328663
75%,47.297337,8.825240
max,47.807957,10.455874


## Retrieve weather forecast data from Open-Meteo API

### Data preparation

The free version of the Open-Meteo API has a limit to the number of API calls per time period (less than 10'000 API calls per day, 5'000 per hour and 600 per minute). Since we request almost 20 data points per location and hourly forecast, one call of a location amounts for roughly 2 calls. Therefore, we limit our dataset to 300 locations in order not to violate these restrictions.

In [11]:
# Create a subset of the dataframe with the first 300 rows
df_subset = df[:300]

In [12]:
# Only keep id, lat ond lon of the subset
df_subset = df_subset[["id", "lat", "lon"]]

In [13]:
# Print the shape of the subset
df_subset.shape

(300, 3)

### API call and transformation

In [14]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

We noticed failed API calls when trying to get data for too many locations in one call. Thus, we split the dataset into batches of 100 locations and make one API call for each batch.

In [15]:
# URL for the Open-Meteo API
url = "https://api.open-meteo.com/v1/forecast"

# Initialize a list to store data for all locations
all_hourly_data = []

# Define a the size of each batch
batch_size = 100

# Add time and datestamp of API call to dataframe
timestamp_apicall = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")

# Loop through all batches and make a request for each batch
for i in range(0, len(df_subset), batch_size):
    latitude = list(df_subset[i:i+batch_size]["lat"])
    longitude = list(df_subset[i:i+batch_size]["lon"])

    item_start = i
    item_end = i + len(latitude) - 1
    batch = int((i + batch_size) / batch_size)

    print("-------------------------")
    print(f"Batch: {batch}")
    print(f"Items: {item_start}-{item_end}")
    print(f"Number of items: {len(latitude)}")

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m",
            "apparent_temperature", "precipitation", "rain", "snowfall",
            "snow_depth", "weather_code", "pressure_msl", "surface_pressure",
            "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high",
            "wind_speed_10m", "wind_gusts_10m", "is_day", "sunshine_duration"
        ],
        "forecast_days": 1,
    }
    
    responses = openmeteo.weather_api(url, params=params)

    print(f"Number of fetched items: {len(responses)}")

    # Loop through all responses and extract data for each location and hourly forecast
    for i, response in enumerate(responses):
        # Process location metadata
        # print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
        # print(f"Elevation {response.Elevation()} m asl")
        # print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
        # print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

        # Define index to be looked up in df_subset
        i_loc = item_start + i

        # Process hourly data for this location
        hourly = response.Hourly()

        # Extract variables (note: needs to be the same order as in request)
        hourly_data = {
            "id": df_subset["id"].iloc[i_loc],
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "lat": df_subset["lat"].iloc[i_loc],
            "lon": df_subset["lon"].iloc[i_loc],
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
            "dew_point_2m": hourly.Variables(2).ValuesAsNumpy(),
            "apparent_temperature": hourly.Variables(3).ValuesAsNumpy(),
            "precipitation": hourly.Variables(4).ValuesAsNumpy(),
            "rain": hourly.Variables(5).ValuesAsNumpy(),
            "snowfall": hourly.Variables(6).ValuesAsNumpy(),
            "snow_depth": hourly.Variables(7).ValuesAsNumpy(),
            "weather_code": hourly.Variables(8).ValuesAsNumpy(),
            "pressure_msl": hourly.Variables(9).ValuesAsNumpy(),
            "surface_pressure": hourly.Variables(10).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(11).ValuesAsNumpy(),
            "cloud_cover_low": hourly.Variables(12).ValuesAsNumpy(),
            "cloud_cover_mid": hourly.Variables(13).ValuesAsNumpy(),
            "cloud_cover_high": hourly.Variables(14).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(15).ValuesAsNumpy(),
            "wind_gusts_10m": hourly.Variables(16).ValuesAsNumpy(),
            "is_day": hourly.Variables(17).ValuesAsNumpy(),
            "sunshine_duration": hourly.Variables(18).ValuesAsNumpy(),
            "timestamp_apicall": timestamp_apicall,
        }

        # Convert to DataFrame and append to list
        all_hourly_data.append(pd.DataFrame(hourly_data))
        
    time.sleep(2)


-------------------------
Batch: 1
Items: 0-99
Number of items: 100
Number of fetched items: 100
-------------------------
Batch: 2
Items: 100-199
Number of items: 100
Number of fetched items: 100
-------------------------
Batch: 3
Items: 200-299
Number of items: 100
Number of fetched items: 100


### Data inspection (weather forecast)

In [16]:
# Print the length of the resulting list
print(len(all_hourly_data))

300


In [17]:
# Concatenate all location data into a single DataFrame
all_hourly_data = pd.concat(all_hourly_data)
print(all_hourly_data.shape)

(7200, 24)


In [18]:
# Describe first half of the columns in the dataframe to decide what data types to use
all_hourly_data.iloc[:, :all_hourly_data.shape[1]//2].describe()

,id,lat,lon,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000
mean,166879.013333,47.118394,8.564355,9.963611,83.141113,7.163215,7.649970,0.280083,0.225972,0.000262,0.004565
std,28470.452458,0.299599,0.757814,3.352885,8.077754,3.310009,4.408439,1.019353,0.900437,0.012482,0.011918
min,22614.000000,46.326090,6.413507,-0.947000,56.000000,-3.152621,-9.467993,0.000000,0.000000,0.000000,0.000000
25%,145859.750000,46.862828,8.194187,7.839625,78.000000,4.856024,4.529517,0.000000,0.000000,0.000000,0.000000
50%,167929.000000,47.233362,8.698550,10.228500,84.000000,7.252944,7.955251,0.000000,0.000000,0.000000,0.000000
75%,188402.250000,47.303267,8.999902,12.482500,89.000000,9.824560,11.098064,0.100000,0.000000,0.000000,0.000000
max,223797.000000,47.602057,10.230199,20.336000,100.000000,15.388756,20.888086,14.900000,14.800001,0.980000,0.070000


In [19]:
# Describe second half of the columns in the dataframe to decide what data types to use
all_hourly_data.iloc[:, all_hourly_data.shape[1]//2:].describe()

,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,wind_speed_10m,wind_gusts_10m,is_day,sunshine_duration
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.00000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000
mean,24.272499,1008.218445,893.844543,69.916389,38.981667,57.36264,4.720417,11.419364,34.034050,0.500000,646.923889
std,33.393726,2.575723,58.622932,33.373337,34.652050,38.72393,20.020611,7.580978,21.215044,0.500035,1299.763062
min,0.000000,1003.299988,761.400452,0.000000,0.000000,0.00000,0.000000,0.360000,2.160000,0.000000,0.000000
25%,2.000000,1006.200012,852.704758,48.000000,0.000000,14.75000,0.000000,5.315336,16.919998,0.000000,0.000000
50%,3.000000,1007.099976,910.774597,83.000000,39.000000,66.00000,0.000000,9.504272,30.239998,0.500000,0.000000
75%,61.000000,1010.400024,945.563507,100.000000,69.000000,99.00000,0.000000,16.167967,47.160000,1.000000,0.000000
max,99.000000,1015.099976,975.819885,100.000000,100.000000,100.00000,100.000000,43.295891,129.240005,1.000000,3600.000000


## Ingestion

### Write to CSV file

In [20]:
# Store data in csv
all_hourly_data.to_csv("../data/processed/weather_forecast_1d_h.csv")

### Create empty SQL table

In [23]:
# Create table if it doesn't exist
table_name = "OPNM_WeatherForecast_1d_H"
query = f"""
    IF OBJECT_ID(N'dbo.{table_name}', N'U') IS NULL
    BEGIN
        CREATE TABLE {table_name} (
            id                      INT         NOT NULL,
            date                    DATETIME    NOT NULL,
            lat                     FLOAT       NOT NULL,
            lon                     FLOAT       NOT NULL,
            temperature_2m          FLOAT       NULL,
            relative_humidity_2m    FLOAT       NULL,
            dew_point_2m            FLOAT       NULL,
            apparent_temperature    FLOAT       NULL,
            precipitation           FLOAT       NULL,
            rain                    FLOAT       NULL,
            snowfall                FLOAT       NULL,
            snow_depth              FLOAT       NULL,
            weather_code            FLOAT       NULL,
            pressure_msl            FLOAT       NULL,
            surface_pressure        FLOAT       NULL,
            cloud_cover             FLOAT       NULL,
            cloud_cover_low         FLOAT       NULL,
            cloud_cover_mid         FLOAT       NULL,
            cloud_cover_high        FLOAT       NULL,
            wind_speed_10m          FLOAT       NULL,
            wind_gusts_10m          FLOAT       NULL,
            is_day                  FLOAT       NULL,
            sunshine_duration       FLOAT       NULL,
            timestamp_apicall       DATETIME    NULL,

            PRIMARY KEY (id, date)
        );
    END
    """

conn = pymssql.connect(server, db_user, db_password, database)
cursor = conn.cursor()
cursor.execute(query)

conn.commit()
conn.close()

### Write to SQL database

In [24]:
# Create connection string for SQLAlchemy
connection_string = f"mssql+pymssql://{db_user}:{db_password}@{server}/{database}"
engine = create_engine(connection_string)

In [25]:
# Ingest data to tabledatabase table
all_hourly_data.to_sql(table_name, con=engine, if_exists='append', index=False)

66